In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

from caen_felib import device

## Connect

In [ ]:
dig = device.connect('dig2://caendgtz-eth-16218.caen.local/')

## Reset

In [ ]:
dig.cmd.reset()

## Example of node browsing

In [ ]:
par_name = 'RecordLengthS'
print(f'Info of parameter {par_name}:')

for child in dig.par.get_node(f'/{par_name}').child_nodes:
    print(f'\t{child.name} == {child.value}')

## Configure digitizer

In [ ]:
dig.par.AcqTriggerSource.value = 'SwTrg'
dig.par.RecordLengthS.value = '4096'
dig.par.PreTriggerS.value = '128'

for ch in dig.ch.child_nodes:
	ch.par.DCOffset.value = f'{ch.name}'  # set dcoffset to channel id

## Configure endpoint

In [ ]:
nch = int(dig.par.numch.value)
reclen = int(dig.par.recordlengths.value)

data_format = [
	{
		'name': 'TIMESTAMP',
		'type': 'U64',
	},
	{
		'name': 'WAVEFORM',
		'type': 'U16',
		'dim': 2,
		'shape': [nch, reclen],
	},
	{
		'name': 'WAVEFORM_SIZE',
		'type': 'U64',
		'dim': 1,
		'shape': [nch],
	},
]

# set_read_data_format returns allocated buffers
data = dig.endpoint.scope.set_read_data_format(data_format)

# set enabled endpoint to activate decode
dig.endpoint.par.ActiveEndpoint.value = 'scope'


## Test waveform plot

In [ ]:
# Start acquisition
dig.cmd.armacquisition()
dig.cmd.swstartacquisition()

# Send trigger and wait for first event
dig.cmd.sendswtrigger()
dig.endpoint.scope.read_data(-1, data)

# Stop acquisition
dig.cmd.disarmacquisition()

# Print timestamp
print(f'Timestamp: {data[0].value}')

# Plot waveforms
for i in range(nch):
    plt.plot(np.arange(0, data[2].value[i]), data[1].value[i])

plt.show()